## Import the Library

In [ ]:
import cv2
import time
import numpy as np

## Load the pre-trained face cascade

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

## Load the cartoon mask image

In [ ]:
cartoon_mask_path = '/Users/simranmodi/Desktop/dora-the-explorer.png'
cartoon_mask = cv2.imread(cartoon_mask_path, cv2.IMREAD_UNCHANGED)

In [ ]:
if cartoon_mask.shape[2] == 3:
    alpha_channel = 255 * np.ones_like(cartoon_mask[:, :, :1], dtype=cartoon_mask.dtype)
    cartoon_mask = np.concatenate([cartoon_mask, alpha_channel], axis=2)


In [ ]:
if cartoon_mask is None:
    print(f"Error: Couldn't load the cartoon mask image at {cartoon_mask_path}")
    exit()

## Open the default camera

In [ ]:
cap = cv2.VideoCapture(0)
# Check if the camera is opened successfully
if not cap.isOpened():
    print("Error: Couldn't open the camera.")
    exit()

In [ ]:
# Define the output file name
output_file = 'simrannmodit7.mp4'

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_file, fourcc, 30, (int(cap.get(3)), int(cap.get(4))))


In [ ]:
# Get the current time
start_time = time.time()

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Check if the frame is successfully read
    if not ret:
        print("Error: Couldn't read a frame from the camera.")
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    # Loop through detected faces and replace them with the cartoon mask
    for (x, y, w, h) in faces:
        # Resize the cartoon mask to match the size of the detected face
        cartoon_mask_resized = cv2.resize(cartoon_mask, (w, h))

        # Replace the face region with the cartoon mask
        frame[y:y+h, x:x+w, :3] = frame[y:y+h, x:x+w, :3] * (1 - cartoon_mask_resized[:, :, 3:] / 255.0) + cartoon_mask_resized[:, :, :3] * (cartoon_mask_resized[:, :, 3:] / 255.0)

    # Display the resulting frame
    cv2.imshow('Cartoon Mask Video', frame)

    # Write the frame to the video file
    out.write(frame)

    # Check if 10 seconds have passed
    if time.time() - start_time > 10:
        break

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and VideoWriter
cap.release()
out.release()

# Destroy any OpenCV windows if you have them open
cv2.destroyAllWindows()
